In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import xgboost as xgb

In [11]:
#Leo la data proveniente de un csv
path_installs = 'datos/installs.csv'
installs = pd.read_csv(path_installs)

In [12]:
installs.shape

(481511, 18)

In [13]:
installs.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.301378e+18
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,adjust.com,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060930e+18
7,2019-04-23 20:55:27.691,1,1494519392962156891,404154463572960372,NaN,False,True,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,06f054c1-608b-437b-a7d7-21d7c5860fe4,app_open,NaN,NaN,5153039437575809299,3.301378e+18
8,2019-04-23 18:14:14.756,1,1494519392962156891,1448747284042458525,NaN,False,False,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,7469505621463131113,3.301378e+18
9,2019-04-23 06:32:40.906,1,1494519392962156891,1465267208368809184,NaN,False,True,6287817205707153877,NaN,4.322426e+18,adjust.com,NaN,e47c9cd6-121e-4c29-91a3-bb6d26afd36f,app_open,NaN,NaN,3744928684799347195,3.301378e+18


In [21]:
# Damos formato a la fecha.
installs['timestamp_format'] = pd.to_datetime(installs['created'])
# Agrego columna fecha
installs['date'] = installs['timestamp_format'].dt.date
# Agrego columna numero dia.
installs['day'] = installs['timestamp_format'].dt.day

In [22]:
installs.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,timestamp_format,date,day
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,...,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18,2019-04-24 06:23:29.495,2019-04-24,24
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,...,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18,2019-04-24 02:06:01.032,2019-04-24,24
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,...,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18,2019-04-20 10:15:36.274,2019-04-20,20
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,...,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18,2019-04-20 21:56:47.151,2019-04-20,20
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,...,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18,2019-04-20 22:40:41.239,2019-04-20,20
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.301378e+18,2019-04-20 18:19:27.485,2019-04-20,20
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,...,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060930e+18,2019-04-20 03:40:21.239,2019-04-20,20
7,2019-04-23 20:55:27.691,1,1494519392962156891,404154463572960372,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,NaN,06f054c1-608b-437b-a7d7-21d7c5860fe4,app_open,NaN,NaN,5153039437575809299,3.301378e+18,2019-04-23 20:55:27.691,2019-04-23,23
8,2019-04-23 18:14:14.756,1,1494519392962156891,1448747284042458525,NaN,False,False,6287817205707153877,NaN,6.208879e+18,...,NaN,NaN,NaN,NaN,NaN,7469505621463131113,3.301378e+18,2019-04-23 18:14:14.756,2019-04-23,23
9,2019-04-23 06:32:40.906,1,1494519392962156891,1465267208368809184,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,NaN,e47c9cd6-121e-4c29-91a3-bb6d26afd36f,app_open,NaN,NaN,3744928684799347195,3.301378e+18,2019-04-23 06:32:40.906,2019-04-23,23


In [24]:
installs18_20 = installs[(installs['day'] >= 18) & (installs['day'] < 21)]
installs19_21 = installs[(installs['day'] >= 19) & (installs['day'] < 22)]
installs20_22 = installs[(installs['day'] >= 20) & (installs['day'] < 23)]
installs21_23 = installs[(installs['day'] >= 21) & (installs['day'] < 24)]
installs22_24 = installs[(installs['day'] >= 22) & (installs['day'] < 25)]

In [43]:
installs18_20['base_time'] = datetime.datetime(2019, 4, 18)
installs18_20['time_to_install'] = (installs18_20['timestamp_format'] - installs18_20['base_time']) / np.timedelta64(1, 's')

installs19_21['base_time'] = datetime.datetime(2019, 4, 19)
installs19_21['time_to_install'] = (installs19_21['timestamp_format'] - installs19_21['base_time']) / np.timedelta64(1, 's')

installs20_22['base_time'] = datetime.datetime(2019, 4, 20)
installs20_22['time_to_install'] = (installs20_22['timestamp_format'] - installs20_22['base_time']) / np.timedelta64(1, 's')

installs21_23['base_time'] = datetime.datetime(2019, 4, 21)
installs21_23['time_to_install'] = (installs21_23['timestamp_format'] - installs21_23['base_time']) / np.timedelta64(1, 's')

installs22_24['base_time'] = datetime.datetime(2019, 4, 22)
installs22_24['time_to_install'] = (installs22_24['timestamp_format'] - installs22_24['base_time']) / np.timedelta64(1, 's')

/home/german/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/german/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/german/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [41]:
installs18_20.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,kind,wifi,trans_id,ip_address,device_language,timestamp_format,date,day,base_time,time_to_install
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,...,app_open,NaN,NaN,8291809486355890410,4.060930e+18,2019-04-20 10:15:36.274,2019-04-20,20,2019-04-18,209736.274
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,...,app_open,NaN,NaN,4006811922873399949,3.301378e+18,2019-04-20 21:56:47.151,2019-04-20,20,2019-04-18,251807.151
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,...,NaN,NaN,NaN,3386455054590810771,3.301378e+18,2019-04-20 22:40:41.239,2019-04-20,20,2019-04-18,254441.239
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,app_open,NaN,NaN,7297343258015606683,3.301378e+18,2019-04-20 18:19:27.485,2019-04-20,20,2019-04-18,238767.485
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,...,app_open,NaN,NaN,4764130939738113581,4.060930e+18,2019-04-20 03:40:21.239,2019-04-20,20,2019-04-18,186021.239
12,2019-04-18 04:00:27.575,1,1494519392962156891,1084778553542198153,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,app_open,NaN,NaN,2980848185112567175,3.301378e+18,2019-04-18 04:00:27.575,2019-04-18,18,2019-04-18,14427.575
13,2019-04-18 08:33:46.588,1,1494519392962156891,6412465357609907698,NaN,False,True,6287817205707153877,NaN,7.538551e+18,...,app_open,NaN,NaN,4934642599625775274,3.301378e+18,2019-04-18 08:33:46.588,2019-04-18,18,2019-04-18,30826.588
14,2019-04-18 20:38:01.661,1,1494519392962156891,8455113036752892959,NaN,False,True,6287817205707153877,NaN,5.544094e+18,...,app_open,NaN,NaN,4312248542004529508,3.301378e+18,2019-04-18 20:38:01.661,2019-04-18,18,2019-04-18,74281.661
15,2019-04-18 18:51:18.157,1,1494519392962156891,7939167172078327528,NaN,False,False,6287817205707153877,NaN,2.355772e+18,...,NaN,NaN,NaN,8713166650905554408,4.060930e+18,2019-04-18 18:51:18.157,2019-04-18,18,2019-04-18,67878.157
22,2019-04-19 04:10:34.820,1,1494519392962156891,2251011940536423208,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,app_open,NaN,NaN,315382286036633979,3.301378e+18,2019-04-19 04:10:34.820,2019-04-19,19,2019-04-18,101434.820
